Четвёртое домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 
В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена из пляшущих человечков.

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.
В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода

In [1]:
import numpy as np
from collections import defaultdict
import editdistance

*TASK 1*: Реализуйте базовый частотный метод по Шерлоку Холмсу:
* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе совсем вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.


In [2]:
def get_text(path):
    "загружает данные для обучения"
    L = []
    with open(path, 'r') as f:
        for line in f:
            if line != '\n':
                L.append(line)
                
    text = ' '.join(L)
    return preproc_text(text)

def preproc_text(text):
    "удаление лишних символов"
    a = ord('а')
    abc = ''.join([chr(i) for i in range(a,a+32)]) + 'ё '
    
    text = text.lower()
    clear_text = ''
    for t in text:
        if t in abc:
            clear_text += t
            
    text = ' '.join(clear_text.split())
    return text


def top_symbols(text, n):
    "возвращает отстортированный словарь и список ngramm по колличеству вхождений"
    counter = defaultdict(int)
    for n_gramm in zip(*[text[i:] for i in range(n)]):
        key = ''
        for c in n_gramm:
            key += c
        counter[key] += 1
        
    counter = {k: v for k, v in sorted(counter.items(), key=lambda item: -item[1])}
    list_counter = list(counter.keys())
    
    return counter, list_counter


def encode(text, map_orig_to_code):
    encode_text = ''
    for t in text:
        encode_text += map_orig_to_code[t]
    return encode_text


def decode_text(train_text, decode_text, n=1):
    ngrams = []
    _, train_list_counter = top_symbols(train_text, n)
    _, test_list_counter = top_symbols(decode_text, n)
    for i in range(0,len(decode_text), n):
        ngrams.append(decode_text[i:i+n])
    ngrams[-1] = decode_text[-n:]
    encode_text = [train_list_counter[test_list_counter.index(n_gram)] for n_gram in ngrams]
    return ''.join(encode_text)

In [3]:
train_text = get_text("data/WarAndPeace.txt")

In [4]:
a = ord('а')
orig = ''.join([chr(i) for i in range(a,a+32)]) + 'ё '
code = ''.join([chr(i) for i in range(1200,1200+34)])

In [5]:
print(orig)
print(code)

абвгдежзийклмнопрстуфхцчшщъыьэюяё 
ҰұҲҳҴҵҶҷҸҹҺһҼҽҾҿӀӁӂӃӄӅӆӇӈӉӊӋӌӍӎӏӐӑ


In [6]:
map_orig_to_code = {}
for o, c in zip(orig, code):
    map_orig_to_code[o] = c

test_text = '''Русское слово пельмени является заимствованием из пермских языков: коми, удм. пельнянь «хлебное ухо»: пель «ухо» + нянь «хлеб»[1][2][3][4][5]. Форма пельмень образовалась под влиянием севернорусского наречия, через которое слово попало в литературный язык. Уральские диалектные формы пермяни, пермени образовались в результате народно-этимологического сближения со словом Пермь.

Некоторые популярные или устаревшие этимологические словари указывают в качестве источника «угро-финские языки» в целом[6], мансийский[7][неавторитетный источник?] и финский[8][неавторитетный источник?] языки.

В. В. Похлёбкин считал, что пельмени пришли в русскую кухню с Урала в конце XIV — начале XV веков[9]. В русских письменных документах уральских населённых пунктов XVII—XVIII веков (с 1679 года) встречаются фамилии, образованные от слова «пельмени»: Пельменев, Пелненев, Пельменников[10]. Историк П. А. Корчагин на основе анализа упоминаний «пельменных» фамилий в документах отметил, что в верхнем Прикамье в 1579 году ещё не было людей, которые зарабатывали изготовлением пельменей[11].

Пельмени в традиционной культуре не были обрядовым блюдом и готовились по праздникам — при встрече гостей и в заговенье'''
test_text = preproc_text(test_text)
encode_text = encode(test_text, map_orig_to_code)
print(test_text)
print(encode_text)

русское слово пельмени является заимствованием из пермских языков коми удм пельнянь хлебное ухо пель ухо нянь хлеб форма пельмень образовалась под влиянием севернорусского наречия через которое слово попало в литературный язык уральские диалектные формы пермяни пермени образовались в результате народноэтимологического сближения со словом пермьнекоторые популярные или устаревшие этимологические словари указывают в качестве источника угрофинские языки в целом мансийскийнеавторитетный источник и финскийнеавторитетный источник языкив в похлёбкин считал что пельмени пришли в русскую кухню с урала в конце начале веков в русских письменных документах уральских населённых пунктов веков с года встречаются фамилии образованные от слова пельмени пельменев пелненев пельменников историк п а корчагин на основе анализа упоминаний пельменных фамилий в документах отметил что в верхнем прикамье в году ещё не было людей которые зарабатывали изготовлением пельменейпельмени в традиционной культуре не были 

In [7]:
#попытка декодировать текст, используя униграммы
decode_text(train_text, encode_text, n=1)

'вуллкао лтаса мотпдоие ьстьорль гнедлрсасниеод ег мовдлкеы ьгякас каде убд мотпиьип ыточиао уыа мотп уыа иьип ыточ шавдн мотпдоип ачвнгаснтнлп маб стеьиеод лосовиавуллкайа инвозеь зовог каравао лтаса мамнта с теровнрувияж ьгяк увнтплкео бентокрияо шавдя мовдьие мовдоие ачвнгаснтелп с вогутпрнро инвабиаэредатайезолкайа лчтефоиеь ла лтасад мовдпиокаравяо мамутьвияо ете улрнвосщео эредатайезолкео лтаснве укнгяснхр с кнзолрсо елразиекн уйвашеилкео ьгяке с юотад днилежлкежионсраверорияж елразиек е шеилкежионсраверорияж елразиек ьгякес с маытцчкеи лзернт зра мотпдоие мвещте с вуллкух куыих л увнтн с каиюо инзнто сокас с вуллкеы мелпдоиияы бакудоирны увнтплкеы инлотциияы муикрас сокас л йабн слрвознхрль шндетее ачвнгасниияо ар лтасн мотпдоие мотпдоиос мотиоиос мотпдоииекас елравек м н кавзнйеи ин алиасо нинтегн умадеиниеж мотпдоиияы шндетеж с бакудоирны ардорет зра с совыиод мвекндпо с йабу оёц ио чята тхбож каравяо гнвнчнряснте егйарастоиеод мотпдоиожмотпдоие с рвнбеюеаииаж кутпруво ио чяте

*TASK 2*: Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п.1, но при помощи биграмм.


In [8]:
#попытка декодировать текст, используя биграммы
decode_text(train_text, encode_text, n=2)

'ве зсти тае на н о кторивилуие е ух кода дтоемл хо нпрь обрианста стлей пао я боойоснышек и сяна н ой этолойоснышетькаадо я ра восбы бс  дтвдоо ми сгоойниовдрлачеказаь есналилобе есклохост иими тае наваз но стуавротрт воотасжей  б оь ниаяндилсввои чапржи нпрой п нпр в пбы бс  дгодо сдишимесптри лиимихд ар жноамскь есназьгоцето еывалнодаов нпрботиоркатео удмерявои од пзан лоебниехар жноамскь ниално делй  ран д я с тязсекни в  инурееньнимку лнеи асже па знноовад лакь акм ым иелогмаисл ковыточт пку лнеыйулеекаавазвоотв  инурерианнеа а ваныахнеутагавпонн ио я ра в пю епго сдизаь вл тсяьеалй  бтв с тшазнолязпои ласта а ве знееречдо к мкиаялсгр вн ерт поиднеерлидрры мкио тнсве  стие алнямиенев э аияие еочлего пбы бс  д мтеу тытае ен н о ктоо я ра вльо я м м а  н о к мрее л кокарео ош ткаивамутлиу аее и  гпоруеных чы  гако я ра ввоерочлегоота егде квнжау г азоднн и с сроятемо ел рдни а онлютсвуоли угнотуажпе торкатеинатоитрожпо пруон иви вниов н о км айя ра в па  эушзвсс мби тмеспт и м солы 

In [9]:
for n in range(1,6):
    levenshtein = editdistance.eval(test_text, decode_text(train_text, encode_text, n=n))
    print("n_gramm = {}, levenshtein dist = {} ".format(n, levenshtein))

n_gramm = 1, levenshtein dist = 849 
n_gramm = 2, levenshtein dist = 895 
n_gramm = 3, levenshtein dist = 882 
n_gramm = 4, levenshtein dist = 868 
n_gramm = 5, levenshtein dist = 883 


In [10]:
print("Длина текста: ", len(test_text))

Длина текста:  1076


Делаем вывод, что н-граммы ухудшают качество. Видимо, слишком маленький тестовый датасет.

*TASK 3*: Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
* предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
* реализуйте и протестируйте его, убедитесь, что результаты улучшились.

*ANS*:

Алгоритм решения задачи:

1) Вычислим матрицу переходных вероятностей $P$ из одного символа в другой. Эта задача уже решена. В пункте 2 мы составили словарь, где каждой биграмме сопоставили сколько раз она встречается в обучающем тексте. Осталось лишь нормировать, чтобы получить вероятности. $P_{a,b}$ - вероятность перехода из символа $a$ в символ $b$.

2) Нам потребуется начальное приближение для функции отображения $f$: закодированного символа в исходный. В качестве такой функции возьмем функцию отображения из пункта 1 на основе униграмм.

3) При помощи $f$ декодируем сообщение и вычислим его правдоподобие: $L(f) = \prod\limits_{k}P_{f(x_k),f(x_{k+1})}$

4) Случайно выбираются два аргумента у функции $f$ и значения функции при этих аргументах меняются местами. Если в результате такой перестановки правдоподобие увеличилось, то изменяем функцию $f$ (чтобы она давала значения, как с перестановкой), если уменьшилось, то изменяем функцию $f$ c вероятностью $\dfrac{L(f_{new})}{L(f)}$

5) Повторяем процедуру до сходимости

In [11]:
def deode_symbol(symbol, train_list_counter, encode_list_counter):
    return train_list_counter[encode_list_counter.index(symbol)]

In [12]:
def mcmc_decode(encode_text, train_list_counter, encode_list_counter):
    ans = []
    for s in encode_text:
        ans.append(deode_symbol(s, train_list_counter, encode_list_counter))
    return ''.join(ans)

In [13]:
def likelihood(text):
    p = 1
    for bi_gramm in zip(*[text[i:] for i in range(2)]):
        try:
            p *= P[bi_gramm[0]+bi_gramm[1]]
        except KeyError:
            p *= min(P.values())
    return p

In [14]:
train_bi_gramm_counter, train_bi_gramm_list_counter = top_symbols(train_text, 2)
train_counter, train_list_counter = top_symbols(train_text, 1)

_, encode_list_counter = top_symbols(encode_text, 1)

In [15]:
N = sum(train_counter.values())
P = train_bi_gramm_counter.copy()

for k, v in P.items():
    P[k] = P[k]/(N/600)

Отнормировали матрицу переходных вероятностей не в единицу, т.к иначе при вычисления правдоподобия быстро упремся в ноль

In [16]:
sum(P.values())

599.9990661376868

In [17]:
decode_text = mcmc_decode(encode_text, train_list_counter, encode_list_counter)
likelihood(decode_text)

3.5417300693003717e-295

In [18]:
L = likelihood(decode_text)
for j in range(100):
    for i in range(100):
        test_list = encode_list_counter.copy()
        idx_0, idx_1 = np.random.randint(0, len(test_list), 2)
        test_list[idx_0], test_list[idx_1] = test_list[idx_1], test_list[idx_0]
        decode_text = mcmc_decode(encode_text, train_list_counter, test_list)
        L_NEW = likelihood(decode_text)
        if L_NEW > L:
            encode_list_counter = test_list
            L = L_NEW
        else:
            p_change = L_NEW/L
            if np.random.uniform(0,1) < p_change:
                encode_list_counter = test_list
                L = L_NEW
    if j % 10 == 0: 
        print("C * likelihood:", L)

C * likelihood: 4.809668649820734e-141
C * likelihood: 1.8099500550286027e+171
C * likelihood: 1.701741214290345e+247
C * likelihood: 5.375790680396495e+261
C * likelihood: 3.176232707530327e+287
C * likelihood: 5.456334119723607e+302
C * likelihood: 5.456334119723607e+302
C * likelihood: 6.2382398030900456e+302
C * likelihood: 6.2382398030900456e+302
C * likelihood: 6.2382398030900456e+302


In [19]:
mcmc_text = mcmc_decode(encode_text, train_list_counter, encode_list_counter)
print(mcmc_text)

русское слово пельмени является заимствованием из пермских языков коми удм пельнянь хлебное ухо пель ухо нянь хлеб жорма пельмень образовалась под влиянием севернорусского наречия через которое слово попало в литературный язык уральские диалектные жормы пермяни пермени образовались в результате народноэтимологического сблищения со словом пермьнекоторые популярные или устаревшие этимологические словари указывают в качестве источника угрожинские языки в целом мансийскийнеавторитетный источник и жинскийнеавторитетный источник языкив в похлёбкин считал что пельмени пришли в русскую кухню с урала в конце начале веков в русских письменных документах уральских населённых пунктов веков с года встречаются жамилии образованные от слова пельмени пельменев пелненев пельменников историк п а корчагин на основе анализа упоминаний пельменных жамилий в документах отметил что в верхнем прикамье в году ефё не было людей которые зарабатывали изготовлением пельменейпельмени в традиционной культуре не были 

In [20]:
levenshtein = editdistance.eval(test_text,mcmc_text)
print("levenshtein dist =", levenshtein)

levenshtein dist = 8


*TASK 4*: Расшифруйте сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏

In [26]:
target_text = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'
print(target_text)

←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏


In [22]:
_, encode_list_counter = top_symbols(target_text, 1)

decode_text = mcmc_decode(target_text, train_list_counter, encode_list_counter)
likelihood(decode_text)

9.14704031136894e-44

In [23]:
L = likelihood(decode_text)
for j in range(500):
    for i in range(100):
        test_list = encode_list_counter.copy()
        idx_0, idx_1 = np.random.randint(0, len(test_list), 2)
        test_list[idx_0], test_list[idx_1] = test_list[idx_1], test_list[idx_0]
        decode_text = mcmc_decode(target_text, train_list_counter, test_list)
        L_NEW = likelihood(decode_text)
        if L_NEW > L:
            encode_list_counter = test_list
            L = L_NEW
        else:
            p_change = L_NEW/L
            if np.random.uniform(0,1) < p_change:
                encode_list_counter = test_list
                L = L_NEW
    if j % 10 == 0: 
        print("C * likelihood:", L)

C * likelihood: 0.0008895775616458849
C * likelihood: 2.4444106864068064e+55
C * likelihood: 7.082304640843486e+66
C * likelihood: 1.411698307322235e+72
C * likelihood: 5.015908432239445e+72
C * likelihood: 9.522416467312183e+72
C * likelihood: 4.798979600042083e+71
C * likelihood: 3.502890118761377e+73
C * likelihood: 2.585291482305958e+73
C * likelihood: 2.585291482305958e+73
C * likelihood: 2.585291482305958e+73
C * likelihood: 3.502890118761377e+73
C * likelihood: 1.9099526081343283e+72
C * likelihood: 1.2198710355580712e+73
C * likelihood: 7.693326662954124e+72
C * likelihood: 9.522416467312183e+72
C * likelihood: 3.502890118761377e+73
C * likelihood: 2.585291482305958e+73
C * likelihood: 4.110409980355971e+72
C * likelihood: 5.015908432239445e+72
C * likelihood: 5.08103741105339e+79
C * likelihood: 2.6162402780177816e+85
C * likelihood: 5.013413227447225e+88
C * likelihood: 8.868392863670773e+89
C * likelihood: 1.0486260297264497e+89
C * likelihood: 3.408509405209821e+90
C * like

In [24]:
decode_text = mcmc_decode(target_text, train_list_counter, encode_list_counter)

In [25]:
print(decode_text)

если вы вимите норшальный или подти норшальный текст у чтого соожбения который легко продитать скорее всего вы все смелали правильно и полудите шаксишальный жалл за послемнее детвертое замание курса хотя конедно я нидего не ожебаю


*Бонус*: какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

*ANS*:
Можем применить такую модель, когда у нас есть файл в неизвестной кодировке, но нам нужно его как-то прочитать.